<a href="https://colab.research.google.com/github/camiloseguel-alt/Speed-Public/blob/main/Version%20CBT%2BSpeed%20Real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Instalar los paquetes necesarios
%pip install --quiet google-cloud-bigquery
!pip install --quiet google-cloud-bigquery pandas openpyxl

from google.cloud import bigquery
from google.colab import auth

# Importar las librerías necesarias
from google.cloud import bigquery
import pandas as pd
from datetime import datetime

# Authenticate to Google Cloud
auth.authenticate_user()

# Conectar al cliente de BigQuery
# Construct a BigQuery client object.
# TODO(developer): Set default project.
# client = bigquery.Client(project="your-project-id")
client = bigquery.Client(project="meli-bi-data")


# Obtener información de la tabla
table_ref = client.dataset("WHOWNER", project="meli-bi-data").table("LK_SHP_SHIPMENTS_HISTORICAL_ND")
table = client.get_table(table_ref)

# Función para mostrar el menú de selección para información histórica/real
def menu_selection_historical():
    while True:
        print("\n**********************************************************************")
        print("POR FAVOR, PROPORCIONE LOS SIGUIENTES PARÁMETROS PARA OBTENER LA INFORMACIÓN HISTORICA/REAL:")
        print("**********************************************************************")
        print("\nSELECCIONE EL PAÍS (SIT_SITE_ID):")
        site_options = ['MCO', 'MLC', 'MLA', 'MLM', 'MLB', 'MEC', 'MLU', 'MPE'] # Reordered alphabetically
        site_options.sort() # Sort the list alphabetically
        for idx, option in enumerate(site_options):
            print(f"{idx + 1}. {option}")

        while True:
            try:
                country_index = int(input("Ingrese el número del país: ")) - 1
                if 0 <= country_index < len(site_options):
                    country = site_options[country_index]
                    break
                else:
                    print("Número de país inválido. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        print("\nSELEccione EL TIPO DE INFORMACIÓN:")
        info_types = ['First Visit', 'Handling', 'Created']
        for idx, option in enumerate(info_types):
            print(f"{idx + 1}. {option}")

        while True:
            try:
                info_type_index = int(input("Ingrese el número del tipo de información: ")) - 1
                if 0 <= info_type_index < len(info_types):
                    info_type = info_types[info_type_index]
                    break
                else:
                    print("Número de tipo de información inválido. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")

        # Año
        while True:
            try:
                year = int(input(f"\nINGRESE EL AÑO (2023 - {datetime.now().year}): "))
                if 2023 <= year <= datetime.now().year:
                    break
                else:
                    print("Fecha inválida, debe ser entre 2023 y el año actual. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        # Mes
        while True:
            try:
                month = int(input(f"\nINGRESE EL MES (1 - 12): "))
                if 1 <= month <= 12:
                    if year == datetime.now().year and month > datetime.now().month:
                         print("Mes inválido para el año actual. Inténtalo de nuevo.")
                    else:
                        break
                else:
                    print("Mes inválido, debe ser entre 1 y 12. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        return country, info_type, year, month

# Función para generar y ejecutar la consulta para información histórica/real
def execute_query_historical(country, info_type, year, month):
    month_str = f"{month:02}"

    date_column_map = {
        'First Visit': 'SHP_DATE_FIRST_VISIT_ID_LTZ',
        'Handling': 'SHP_DATE_HANDLING_ID_LTZ',
        'Created': 'SHP_DATE_CREATED_ID'
    }

    date_column = date_column_map[info_type]

    print(f"\nExecuting query for Historical/Real Info with:")
    print(f"  Country: {country}")
    print(f"  Info Type: {info_type}")
    print(f"  Date Column: {date_column}")
    print(f"  Year: {year}")
    print(f"  Month: {month}")


    query = f"""
    SELECT
        SIT_SITE_ID,
        COUNT(SHP_SHIPMENT_ID) AS SHP,
        SUM (SHP_QUANTITY) AS SIS,
        CASE
            WHEN SIT_SITE_ID = 'MCO' AND SHP_ADD_STATE_ID_BUYER = 'CO-DC' THEN 'CO-BDC'
            ELSE SHP_ADD_STATE_ID_BUYER
        END AS SHP_ADD_STATE_ID_BUYER,
        CASE
            WHEN SIT_SITE_ID = 'MCO' AND SHP_ADD_STATE_ID_SELLER = 'CO-DC' THEN 'CO-BDC'
            ELSE SHP_ADD_STATE_ID_SELLER
        END AS SHP_ADD_STATE_ID_SELLER,
        CASE
            WHEN SUBSTR(SHP_ADD_STATE_ID_SELLER, 1, 2) IN ('CN', 'HK', 'US') THEN 'CBT'
            ELSE 'LOCAL'
        END AS ORIGEN_CBT,
        EXTRACT(YEAR FROM {date_column}) AS YEAR,
        EXTRACT(MONTH FROM {date_column}) AS MONTH,
        CASE
            WHEN SHP_PICKING_TYPE_ID = 'drop_off' THEN 'DS'
            WHEN SHP_PICKING_TYPE_ID = 'xd_drop_off' THEN 'XD'
            WHEN SHP_PICKING_TYPE_ID = 'fulfillment' THEN 'FUL'
            WHEN SHP_PICKING_TYPE_ID = 'cross_docking' THEN 'XD'
            WHEN SHP_PICKING_TYPE_ID = 'self_service' THEN 'FLEX'
            ELSE NULL
        END AS SHP_PICKING_TYPE_ID,
        CASE
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 0 THEN 'SD'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 1 THEN 'ND'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 2 THEN '2D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 3 THEN '3D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 4 THEN '4D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 5 THEN '5D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 6 THEN '6D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 7 THEN '7D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 8 THEN '8D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 9 THEN '9D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS > 9 THEN '9DP'
            ELSE NULL
        END AS SHP_LEAD_TIME_NATURAL_DAYS
    FROM `meli-bi-data.WHOWNER.LK_SHP_SHIPMENTS_HISTORICAL_ND`
    WHERE
        SIT_SITE_ID = '{country}' AND
        LOWER(SHP_FLOW_TYPE) = 'forward' AND
        SHP_SOURCE_ID = 'MELI' AND
        EXTRACT(YEAR FROM {date_column}) = {year} AND
        EXTRACT(MONTH FROM {date_column}) = {month} AND
        SHP_LEAD_TIME_NATURAL_DAYS IS NOT NULL
    GROUP BY
        SIT_SITE_ID,
        SHP_ADD_STATE_ID_BUYER,
        SHP_ADD_STATE_ID_SELLER,
        ORIGEN_CBT,
        YEAR,
        MONTH,
        SHP_PICKING_TYPE_ID,
        SHP_LEAD_TIME_NATURAL_DAYS
    """

    df = client.query(query).result().to_dataframe()

    print(f"Number of rows returned by the query: {len(df)}")
    print(f"Data type of 'SIS' column: {df['SIS'].dtype}")
    # print("\nDataFrame columns after fetching from BigQuery:")
    # print(df.columns)
    # Remove the display of the first 5 rows after fetching from BigQuery
    # display(df.head())


    # Rename Year and Month columns based on info_type
    rename_map = {}
    if info_type == 'First Visit':
        rename_map['YEAR'] = 'YEAR_FV'
        rename_map['MONTH'] = 'MONTH_FV'
        year_col = 'YEAR_FV'
        month_col = 'MONTH_FV'
    elif info_type == 'Handling':
        rename_map['YEAR'] = 'YEAR_HT'
        rename_map['MONTH'] = 'MONTH_HT'
        year_col = 'YEAR_HT'
        month_col = 'MONTH_HT'
    elif info_type == 'Created':
        rename_map['YEAR'] = 'YEAR_C'
        rename_map['MONTH'] = 'MONTH_C'
        year_col = 'YEAR_C'
        month_col = 'MONTH_C'


    df.rename(columns=rename_map, inplace=True)

    # Explicitly convert 'SIS' to numeric, coercing errors will turn unparseable values into NaN
    df['SIS'] = pd.to_numeric(df['SIS'], errors='coerce')


    df = df[df['SHP_PICKING_TYPE_ID'].notnull()]  # Ignorar no estándar

    # Print unique values in SHP_LEAD_TIME_NATURAL_DAYS before standardization to debug
    # print("Unique values in SHP_LEAD_TIME_NATURAL_DAYS before standardization:")
    # The SHP_LEAD_TIME_NATURAL_DAYS column is now the result of the CASE statement
    # print(df['SHP_LEAD_TIME_NATURAL_DAYS'].unique())


    # Calculate column SHARE_PT (J) based on the provided formula
    # SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;$D2;$E:$E;$E2;$F:$F;$F2;$G:$G;$G2)/SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2)
    # H -> SHP, A -> SIT_SITE_ID, B -> YEAR_FV/HT/C, C -> MONTH_FV/HT/C, D -> SHP_PICKING_TYPE_ID, E -> SHP_ADD_STATE_ID_BUYER, F -> SHP_ADD_STATE_ID_SELLER, G -> SHP_LEAD_TIME_NATURAL_DAYS

    # Calculate the denominator first (sum of SHP for each SIT_SITE_ID, Year, Month group)
    denominator = df.groupby(['SIT_SITE_ID', year_col, month_col])['SHP'].transform('sum')

    # Calculate the numerator (sum of SHP for each group defined by all relevant columns)
    numerator = df.groupby(['SIT_SITE_ID', year_col, month_col, 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'ORIGEN_CBT', 'SHP_LEAD_TIME_NATURAL_DAYS'])['SHP'].transform('sum')

    # Calculate SHARE_PT, handling division by zero
    df['SHARE_PT'] = numerator / denominator
    df['SHARE_PT'] = df['SHARE_PT'].fillna(0) # Replace NaN resulting from division by zero with 0


    # Calculate column SPEED_PT based on the provided formula
    # H2/SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;D2)
    # H -> SHP, A -> SIT_SITE_ID, B -> YEAR_FV/HT/C, C -> MONTH_FV/HT/C, D -> SHP_PICKING_TYPE_ID
    speed_pt_denominator = df.groupby(['SIT_SITE_ID', year_col, month_col, 'SHP_PICKING_TYPE_ID'])['SHP'].transform('sum')
    df['SPEED_PT'] = df['SHP'] / speed_pt_denominator
    df['SPEED_PT'] = df['SPEED_PT'].fillna(0) # Replace NaN resulting from division by zero with 0


    # Calculate column SPEED_ALL_NET based on the provided formula
    # K2*SUMIFS($J:$J;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;$D2)
    # J -> SHARE_PT, A -> SIT_SITE_ID, B -> YEAR_FV/HT/C, C -> MONTH_FV/HT/C, D -> SHP_PICKING_TYPE_ID
    speed_all_net_sumifs = df.groupby(['SIT_SITE_ID', year_col, month_col, 'SHP_PICKING_TYPE_ID'])['SHARE_PT'].transform('sum')
    df['SPEED_ALL_NET'] = df['SPEED_PT'] * speed_all_net_sumifs


    # print("\nDataFrame columns after calculating new columns:")
    # print(df.columns)
    # Remove the display of the first 5 rows after calculating new columns
    # display(df.head())


    # Reorder columns
    if info_type == 'First Visit':
        ordered_columns = ['SIT_SITE_ID', 'YEAR_FV', 'MONTH_FV', 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'ORIGEN_CBT', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']
    elif info_type == 'Handling':
         ordered_columns = ['SIT_SITE_ID', 'YEAR_HT', 'MONTH_HT', 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'ORIGEN_CBT', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']
    elif info_type == 'Created':
        ordered_columns = ['SIT_SITE_ID', 'YEAR_C', 'MONTH_C', 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'ORIGEN_CBT', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']

    # Ensure all required columns exist in the DataFrame before reordering
    missing_columns = [col for col in ordered_columns if col not in df.columns]
    if missing_columns:
        print(f"Warning: The following columns are missing from the DataFrame and cannot be reordered: {missing_columns}")
        # Proceed with reordering only the existing columns
        ordered_columns = [col for col in ordered_columns if col in df.columns]


    df = df[ordered_columns]

    return df

# Función para mostrar el menú de selección para información del plan
def menu_selection_plan():
    while True:
        print("\n**********************************************************************")
        print("POR FAVOR, PROPORCIONE LOS SIGUIENTES PARÁMETROS PARA OBTENER LA INFORMACIÓN DEL PLAN:")
        print("**********************************************************************")

        # Prompt for SPIDER_ID first
        while True:
            try:
                spider_id = int(input("\nINGRESE EL ID DEL PLAN (CAMPO SPIDER_ID): "))
                break
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número entero para el SPIDER_ID.")

        # Get unique sites for the entered spider_id
        sites_query = f"SELECT DISTINCT SITE FROM `meli-bi-data.SBOX_NETWORKD.SPIDER_HISTORY` WHERE SPIDER_ID = {spider_id} ORDER BY SITE"
        sites_df = client.query(sites_query).result().to_dataframe()
        site_options = sites_df['SITE'].tolist()

        if not site_options:
            print(f"No se encontraron países para el SPIDER_ID: {spider_id}. Por favor, intente con un SPIDER_ID diferente.")
            continue # Restart the loop to ask for SPIDER_ID again
        elif len(site_options) > 1:
            print(f"Warning: Multiple countries found for SPIDER_ID {spider_id}. Please select one.")
            # Display the options and let the user select
            print("\nSELECCIONE EL PAÍS (SITE):")
            for idx, option in enumerate(site_options):
                print(f"{idx + 1}. {option}")
            while True:
                try:
                    country_index = int(input("Ingrese el número del país: ")) - 1
                    if 0 <= country_index < len(site_options):
                        country = site_options[country_index]
                        break
                    else:
                        print("Número de país inválido. Inténtalo de nuevo.")
                except ValueError:
                    print("Entrada inválida. Por favor, ingrese un número.")
        else:
            # Only one country found, ask for confirmation with the new text
            country = site_options[0]
            while True:
                confirm = input(f"El ID seleccionado corresponde a {country} es ¿correcto? (si/no): ").lower()
                if confirm == 'si':
                    break # Exit the confirmation loop
                elif confirm == 'no':
                    print("Por favor, ingrese el ID del plan nuevamente.")
                    break # Exit the confirmation loop to re-enter spider_id
                else:
                    print("Respuesta inválida. Por favor, ingrese 'si' o 'no'.")

            if confirm == 'no':
                continue # Restart the outer loop to ask for SPIDER_ID again


        # Get available years and months for the selected spider_id and country
        year_month_query = f"""
        SELECT DISTINCT YEAR_NUMBER, MONTH_NUMBER
        FROM `meli-bi-data.SBOX_NETWORKD.SPIDER_HISTORY`
        WHERE SITE = '{country}'
          AND SPIDER_ID = {spider_id}
        ORDER BY YEAR_NUMBER, MONTH_NUMBER
        """
        year_month_df = client.query(year_month_query).result().to_dataframe()

        if year_month_df.empty:
             print(f"No se encontraron datos de año/mes para el país: {country} y SPIDER_ID: {spider_id}. Por favor, intente con un SPIDER_ID o país diferente.")
             continue # Restart the loop

        print(f"\nAños y Meses disponibles para el país: {country} y SPIDER_ID: {spider_id}:")
        for index, row in year_month_df.iterrows():
            # Convert to int before printing to remove decimal
            print(f"  - Año: {int(row['YEAR_NUMBER'])}, Mes: {int(row['MONTH_NUMBER'])}")


        # Año
        while True:
            try:
                year = int(input(f"\nINGRESE EL AÑO: "))
                # Check if the entered year is in the available years for the selected spider_id and country
                if year in year_month_df['YEAR_NUMBER'].unique():
                    break
                else:
                    print("Año inválido. Por favor, ingrese un año de la lista disponible.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        # Mes
        while True:
            try:
                month = int(input(f"\nINGRESE EL MES: "))
                # Check if the entered month is in the available months for the selected year, spider_id, and country
                if month in year_month_df[year_month_df['YEAR_NUMBER'] == year]['MONTH_NUMBER'].unique():
                    break
                else:
                    print(f"Mes inválido para el año {year}. Por favor, ingrese un mes de la lista disponible para este año.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        return country, spider_id, year, month

# Función para generar y ejecutar la consulta para información del plan
def execute_query_plan(country, spider_id, year, month):
    print(f"\nExecuting query for Plan Info with:")
    print(f"  Country: {country}")
    print(f"  Spider ID: {spider_id}")
    print(f"  Year: {year}")
    print(f"  Month: {month}")

    query = f"""
    SELECT
        SPIDER_ID,
        SITE,
        YEAR_NUMBER,
        MONTH_NUMBER,
        PICKING_TYPE,
        CASE
            WHEN SITE = 'MCO' AND BUYER_STATE = 'CO-DC' THEN 'CO-BDC'
            ELSE BUYER_STATE
        END AS BUYER_STATE,
        CASE
            WHEN SITE = 'MCO' AND SELLER_STATE = 'CO-DC' THEN 'CO-BDC'
            ELSE SELLER_STATE
        END AS SELLER_STATE,
         CASE
            WHEN SUBSTR(SELLER_STATE, -2) IN ('CN', 'HK', 'US', 'CH') THEN 'CBT'
            ELSE 'LOCAL'
        END AS ORIGEN_CBT,
        SUM(SHIPMENTS) AS SHP,
        SUM(SIS) AS SIS
    FROM `meli-bi-data.SBOX_NETWORKD.SPIDER_HISTORY`
    WHERE
        SITE = '{country}' AND
        SPIDER_ID = {spider_id} AND
        YEAR_NUMBER = {year} AND
        MONTH_NUMBER = {month} AND
        LOWER(FLOW_TYPE) = 'forward'
    GROUP BY
        SPIDER_ID,
        SITE,
        YEAR_NUMBER,
        MONTH_NUMBER,
        PICKING_TYPE,
        BUYER_STATE,
        SELLER_STATE,
        ORIGEN_CBT
    """

    df = client.query(query).result().to_dataframe()

    print(f"Number of rows returned by the plan query: {len(df)}")
    # print("\nDataFrame columns after fetching from BigQuery (Plan):")
    # print(df.columns)
    # Remove the display of the first 5 rows after fetching from BigQuery
    # display(df.head())

    # Perform calculations for SHARE_PT, SPEED_PT, and SPEED_ALL_NET
    # Ensure 'SHP' is numeric
    df['SHP'] = pd.to_numeric(df['SHP'], errors='coerce')
    df = df.dropna(subset=['SHP']) # Drop rows where SHP could not be converted

    # Calculate the denominator for SHARE_PT (sum of SHP for each Site, Year, Month group)
    # Assuming the relevant grouping columns are SITE, YEAR_NUMBER, MONTH_NUMBER
    denominator_share_pt = df.groupby(['SITE', 'YEAR_NUMBER', 'MONTH_NUMBER'])['SHP'].transform('sum')

    # Calculate the numerator for SHARE_PT (sum of SHP for each group defined by all relevant columns)
    # Assuming the relevant grouping columns are SITE, YEAR_NUMBER, MONTH_NUMBER, PICKING_TYPE, BUYER_STATE, SELLER_STATE
    numerator_share_pt = df.groupby(['SITE', 'YEAR_NUMBER', 'MONTH_NUMBER', 'PICKING_TYPE', 'BUYER_STATE', 'SELLER_STATE', 'ORIGEN_CBT'])['SHP'].transform('sum')

    # Calculate SHARE_PT, handling division by zero
    df['SHARE_PT'] = numerator_share_pt / denominator_share_pt
    df['SHARE_PT'] = df['SHARE_PT'].fillna(0) # Replace NaN resulting from division by zero with 0


    # Calculate column SPEED_PT based on the provided formula
    # H2/SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;D2)
    # H -> SHP, A -> SITE, B -> YEAR_NUMBER, C -> MONTH_NUMBER, D -> PICKING_TYPE
    speed_pt_denominator = df.groupby(['SITE', 'YEAR_NUMBER', 'MONTH_NUMBER', 'PICKING_TYPE'])['SHP'].transform('sum')
    df['SPEED_PT'] = df['SHP'] / speed_pt_denominator
    df['SPEED_PT'] = df['SPEED_PT'].fillna(0) # Replace NaN resulting from division by zero with 0


    # Calculate column SPEED_ALL_NET based on the provided formula
    # K2*SUMIFS($J:$J;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;$D2)
    # J -> SHARE_PT, A -> SITE, B -> YEAR_NUMBER, C -> MONTH_NUMBER, D -> SHP_PICKING_TYPE_ID
    speed_all_net_sumifs = df.groupby(['SITE', 'YEAR_NUMBER', 'MONTH_NUMBER', 'PICKING_TYPE'])['SHARE_PT'].transform('sum')
    df['SPEED_ALL_NET'] = df['SPEED_PT'] * speed_all_net_sumifs

    # Convert YEAR_NUMBER, MONTH_NUMBER, SHP, and SIS to integers
    df['YEAR_NUMBER'] = df['YEAR_NUMBER'].astype(int)
    df['MONTH_NUMBER'] = df['MONTH_NUMBER'].astype(int)
    df['SHP'] = df['SHP'].astype(int)
    df['SIS'] = df['SIS'].astype(int)


    # Reorder columns for the plan data
    ordered_columns_plan = ['SPIDER_ID', 'SITE', 'YEAR_NUMBER', 'MONTH_NUMBER', 'PICKING_TYPE', 'BUYER_STATE', 'SELLER_STATE', 'ORIGEN_CBT', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']

    # Ensure all required columns exist in the DataFrame before reordering
    missing_columns_plan = [col for col in ordered_columns_plan if col not in df.columns]
    if missing_columns_plan:
        print(f"Warning: The following columns are missing from the Plan DataFrame and cannot be reordered: {missing_columns_plan}")
        # Proceed with reordering only the existing columns
        ordered_columns_plan = [col for col in ordered_columns_plan if col in df.columns]

    df = df[ordered_columns_plan]


    return df


# Ejecutar los menús y las consultas, y guardar en diferentes hojas
country_hist, info_type, year_hist, month_hist = menu_selection_historical()
df_historical = execute_query_historical(country_hist, info_type, year_hist, month_hist)

# Modified to prompt for SPIDER_ID first and then use it for subsequent filters
country_plan, spider_id_plan, year_plan, month_plan = menu_selection_plan()

# Check if df_plan was successfully created before attempting to save
df_plan = execute_query_plan(country_plan, spider_id_plan, year_plan, month_plan)


# Create the pivot table
# Ensure the correct Year and Month columns are used based on info_type
year_col_pivot = ''
month_col_pivot = ''
if info_type == 'First Visit':
    year_col_pivot = 'YEAR_FV'
    month_col_pivot = 'MONTH_FV'
elif info_type == 'Handling':
    year_col_pivot = 'YEAR_HT'
    month_col_pivot = 'MONTH_HT'
elif info_type == 'Created':
    year_col_pivot = 'YEAR_C'
    month_col_pivot = 'MONTH_C'

# Check if the required columns exist in df_historical before creating the pivot table
required_pivot_columns = ['ORIGEN_CBT', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SIT_SITE_ID', year_col_pivot, month_col_pivot, 'SHP', 'SHP_PICKING_TYPE_ID']
if all(col in df_historical.columns for col in required_pivot_columns):
    pivot_table = pd.pivot_table(
        df_historical,
        values='SHP',
        index=['SIT_SITE_ID', year_col_pivot, month_col_pivot, 'SHP_PICKING_TYPE_ID'],
        columns='SHP_LEAD_TIME_NATURAL_DAYS',
        aggfunc='sum'
    )

    # Define the desired column order
    desired_column_order = ['SD', 'ND', '2D', '3D', '4D', '5D', '6D', '7D', '8D', '9D', '9DP']

    # Reindex the columns of the pivot table based on the desired order
    # Get the existing columns that are in the desired order
    existing_columns_in_order = [col for col in desired_column_order if col in pivot_table.columns]
    # Get any existing columns that are not in the desired order
    remaining_columns = [col for col in pivot_table.columns if col not in desired_column_order]
    # Combine them: ordered columns first, then remaining columns
    reordered_columns = existing_columns_in_order + remaining_columns
    pivot_table = pivot_table[reordered_columns]

    # Calculate the 'ALL_NTW' row
    all_ntw_row_data = {}
    total_shp_historical = df_historical['SHP'].sum() # Calculate the total SHP from the original historical data

    if total_shp_historical != 0:
        for col in pivot_table.columns:
            # Sum of SHP for the current lead time column in the pivot table
            column_sum_shp = pivot_table[col].sum()
            # Calculate the percentage relative to the total SHP in the historical data
            all_ntw_row_data[col] = (column_sum_shp / total_shp_historical) * 100
    else:
        # If total_shp_historical is 0, all values in the ALL_NTW row should be 0
        for col in pivot_table.columns:
            all_ntw_row_data[col] = 0


    # Create a DataFrame for the 'ALL_NTW' row
    # The index needs to match the structure of the original pivot table index, but with 'ALL_NTW' as the first level
    all_ntw_index = pd.MultiIndex.from_tuples([('ALL_NTW', '', '', '')],
                                              names=['SIT_SITE_ID', year_col_pivot, month_col_pivot, 'SHP_PICKING_TYPE_ID'])
    all_ntw_df = pd.DataFrame(all_ntw_row_data, index=all_ntw_index)

    # Reindex the 'ALL_NTW' row to match the pivot table columns order
    all_ntw_df = all_ntw_df.reindex(columns=pivot_table.columns)

    # Append the 'ALL_NTW' row to the pivot table
    pivot_table_with_all_ntw = pd.concat([pivot_table, all_ntw_df])


    # Calculate percentage of column total for the full table (including ALL_NTW row)
    # We will apply percentage calculation to the entire concatenated table
    pivot_table_percentage = pivot_table_with_all_ntw.apply(lambda x: x / x.sum() * 100, axis=1)


    # Guardar resultados a un archivo Excel con múltiples hojas
    with pd.ExcelWriter("info.xlsx") as writer:
        df_historical.to_excel(writer, sheet_name="Info Real", index=False)
        # Only attempt to save df_plan if it exists and is not empty
        if df_plan is not None and not df_plan.empty:
            df_plan.to_excel(writer, sheet_name="Info Plan", index=False)
        pivot_table_percentage.to_excel(writer, sheet_name="Pivot Table") # Save the pivot table with percentage

    print("\nResultados guardados en 'info.xlsx' con las hojas 'Info Real', 'Info Plan' y 'Pivot Table'.")
    print("\nPivot Table DataFrame:")
    display(pivot_table_percentage)


else:
    print("\nCould not create pivot table. Missing required columns in the historical data.")
    # Save the Excel file without the pivot table if it couldn't be created
    with pd.ExcelWriter("info.xlsx") as writer:
        df_historical.to_excel(writer, sheet_name="Info Real", index=False)
        if df_plan is not None and not df_plan.empty:
            df_plan.to_excel(writer, sheet_name="Info Plan", index=False)
    print("\nResultados guardados en 'info.xlsx' con las hojas 'Info Real' e 'Info Plan'.")


**********************************************************************
POR FAVOR, PROPORCIONE LOS SIGUIENTES PARÁMETROS PARA OBTENER LA INFORMACIÓN HISTORICA/REAL:
**********************************************************************

SELECCIONE EL PAÍS (SIT_SITE_ID):
1. MCO
2. MEC
3. MLA
4. MLB
5. MLC
6. MLM
7. MLU
8. MPE
Ingrese el número del país: 1

SELEccione EL TIPO DE INFORMACIÓN:
1. First Visit
2. Handling
3. Created
Ingrese el número del tipo de información: 2

INGRESE EL AÑO (2023 - 2025): 2024

INGRESE EL MES (1 - 12): 10

Executing query for Historical/Real Info with:
  Country: MCO
  Info Type: Handling
  Date Column: SHP_DATE_HANDLING_ID_LTZ
  Year: 2024
  Month: 10
Number of rows returned by the query: 10022
Data type of 'SIS' column: object

**********************************************************************
POR FAVOR, PROPORCIONE LOS SIGUIENTES PARÁMETROS PARA OBTENER LA INFORMACIÓN DEL PLAN:
**********************************************************************



SHP_LEAD_TIME_NATURAL_DAYS                               SD         ND  \
SIT_SITE_ID YEAR_HT MONTH_HT SHP_PICKING_TYPE_ID                         
MCO         2024    10       DS                    0.001027   3.326279   
                             FLEX                 45.814773  46.661145   
                             FUL                   9.631761  50.606173   
                             XD                    0.000404  14.939457   
ALL_NTW                                           11.248729  27.952925   

SHP_LEAD_TIME_NATURAL_DAYS                               2D         3D  \
SIT_SITE_ID YEAR_HT MONTH_HT SHP_PICKING_TYPE_ID                         
MCO         2024    10       DS                     9.28749   11.03692   
                             FLEX                  5.409892   1.715018   
                             FUL                  22.890408   8.539904   
                             XD                   32.172128  22.931863   
ALL_NTW                                           23.277523  14.847581   

SHP_LEAD_TIME_NATURAL_DAYS                               4D        5D  \
SIT_SITE_ID YEAR_HT MONTH_HT SHP_PICKING_TYPE_ID                        
MCO         2024    10       DS                   10.411316  8.640314   
                             FLEX                  0.304861  0.040718   
                             FUL                    3.56055  1.810059   
                             XD                    12.89665  7.273884   
ALL_NTW                                            8.246642  4.772918   

SHP_LEAD_TIME_NATURAL_DAYS                              6D        7D  \
SIT_SITE_ID YEAR_HT MONTH_HT SHP_PICKING_TYPE_ID                       
MCO         2024    10       DS                   6.685431  4.284716   
                             FLEX                 0.014965  0.009744   
                             FUL                   1.08238   0.68131   
                             XD                   3.327399  1.782982   
ALL_NTW                                           2.422325  1.367415   

SHP_LEAD_TIME_NATURAL_DAYS                              8D        9D       9DP  
SIT_SITE_ID YEAR_HT MONTH_HT SHP_PICKING_TYPE_ID                                
MCO         2024    10       DS                   2.979064  2.905101  40.44234  
                             FLEX                  0.00696  0.002436  0.019489  
                             FUL                  0.353686   0.22135  0.622419  
                             XD                   1.023691  0.653128  2.998414  
ALL_NTW                                           0.813086  0.586155    4.4647